In [27]:
###carga las noticias relacionadas a la violencia de genero

import glob
import pandas as pd
csv_files = glob.glob('1-4mes/noticias relacionadas a la violencia de genero/*.csv')



list_data = []
  
for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)
#Para chequear que todo está bien, mostramos la list_data por consola

 
data1 = pd.concat(list_data,ignore_index=True)
data1.drop('Unnamed: 0', axis=1, inplace=True)




In [28]:
csv_files = glob.glob('5-8mes/noticias relacionadas a la violencia de genero/*.csv')



list_data = []
  
for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)
#Para chequear que todo está bien, mostramos la list_data por consola

 
data2 = pd.concat(list_data,ignore_index=True)
data2.drop('Unnamed: 0', axis=1, inplace=True)


In [29]:
csv_files = glob.glob('9-12mes/noticias relacionadas a la violencia de genero/*.csv')



list_data = []
  
for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)
#Para chequear que todo está bien, mostramos la list_data por consola

 
data3 = pd.concat(list_data,ignore_index=True)
data3.drop('Unnamed: 0', axis=1, inplace=True)

In [30]:
data = pd.concat([data1, data2, data3], axis=0)
data.reset_index(drop='True',inplace=True)


In [31]:
data

,title,text
0,Desde mitigar los efectos del cambio climático...,La Fundación dio a conocer ayer las ganadoras ...
1,Desde mitigar los efectos del cambio climático...,La Fundación dio a conocer ayer las ganadoras ...
2,Autoridades preparan arremetida contra los rui...,Una fuerte arremetida están preparando las aut...
3,FOSIS y Fiscalía Regional renovaron convenio p...,Un total de 30 usuarias serán beneficiadas a t...
4,Fiscalía confirma que investiga secuestro con ...,"""Ayúdame, estoy amenazada llama al 133, me tie..."
...,...,...
480,Denuncia por abuso genera furia entre vecinos ...,"En destrozos, saqueo y un principio de incendi..."
481,Buscan otras víctimas de aberrante caso sexual,Con el trabajo conjunto de la Policía de Inves...
482,Ancud: dictan prisión por aberrantes ultrajes ...,En prisión preventiva quedó un sujeto tras ser...
483,Sujeto fue detenido por violación y almacenami...,Detectives de la Brigada Investigadora de Deli...


In [ ]:
#data.to_csv('noticias relacionadas a la violencia de genero.csv')

In [49]:
noticias = data.text.values.tolist()

In [50]:
import spacy
from spacy.matcher import Matcher

In [51]:
nlp = spacy.load("es_core_news_md")

In [52]:
matcher = Matcher(nlp.vocab)

In [54]:
import numpy as np
import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [55]:
pattern_1 = [{"POS": "NOUN"},{"LOWER": "de"}, {"POS": "NOUN"}]
matcher.add("NOUN-de-NOUN", [pattern_1])

pattern_2 = [{"POS": "NOUN"}, {"POS": "ADJ"}]
matcher.add("NOUN-ADJ", [pattern_2])

In [56]:
def text_to_list(noticia):
    list_of_words = []
    
    try:
        doc = nlp(noticia)

        for token in doc:
            if (token.pos_=="NOUN"):
                list_of_words.append(token.text)

        for ent in doc.ents:
            if (ent.label_ == "PER" and " " in ent.text):
                list_of_words.append(ent.text)

        matches = matcher(doc)

        for match_id, start, end in matches:
            span = doc[start:end]  # The matched span
            list_of_words.append(span.text)
    
    except Exception as e: 
        print(noticia)
        print(e)
    
    return list_of_words

In [57]:
noticias_procesadas = []

for index, noticia in enumerate(noticias):
    print(index)
    noticia_procesada = text_to_list(noticia)
    noticias_procesadas.append(noticia_procesada)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [59]:
print(noticias_procesadas[5])

['movilización', 'mujeres', 'agrupaciones', 'cuanto', 'actividades', 'años', 'convocatoria', 'grupos', 'mujeres', 'capoeira', 'violencia', 'denuncias', 'violencia', 'falla', 'anticonceptivos', 'consultorios', 'cosas', 'realidad', 'nivel', 'desaparición', 'año', 'pese', 'concentración', 'personas', 'resguardos', 'pandemia', 'uso', 'mascarilla', 'distancia', 'exigencias', 'bandera', 'lucha', 'aborto', 'derechos', 'fin', 'criminalización', 'migración', 'derecho', 'trabajo', 'vida', 'violencia', 'reconocimiento', 'dignidad', 'trabajo', 'actividades', 'programas', 'conmemoración', 'día', 'mujer', 'marcha', 'plaza', 'terminal', 'buses', 'reunión', 'horas', 'salida', 'marcha', '18.30', 'cuenta', 'fila', 'participantes', 'cuadras', 'actividad', 'cierre', 'edición', 'forma', 'alteración', 'orden', 'actividades', 'mediodía', 'intervención', 'organización', 'favor', 'educación', 'derecho', 'aborto', 'defensa', 'derechos', 'vida', 'violencia', 'apruebo', 'constitución', 'años', 'artista', 'plaza',

In [60]:
# Create Dictionary
id2word = corpora.Dictionary(noticias_procesadas)
id2word

In [61]:
id2word[179]

'plataformas'

In [62]:
texts = noticias_procesadas

In [63]:
dataset = [id2word.doc2bow(noticia_procesada) for noticia_procesada in noticias_procesadas]
print(dataset[:5])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 7), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 2), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 5), (60, 1), (61, 2), (62, 1), (63, 1), (64, 3), (65, 1), (66, 1), (67, 2), (68, 2), (69, 1), (70, 2), (71, 1), (72, 1), (73, 3), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 4), (80, 1), (81, 1), (82, 1), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 6), (109, 2), (110, 2)

In [64]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=dataset,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [66]:
from imp import reload

In [67]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, dataset, id2word)
vis

/home/fernando/miniconda3/envs/tal/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.198362  0.088797       1        1  39.898933
4      0.012741 -0.211373       2        1  19.115275
3     -0.112949 -0.031393       3        1  16.615341
1      0.193441  0.006506       4        1  15.797335
0      0.105129  0.147463       5        1   8.573116, topic_info=              Term        Freq       Total Category  logprob  loglift
41            años  749.000000  749.000000  Default  30.0000  30.0000
948          abuso  289.000000  289.000000  Default  29.0000  29.0000
1335  abuso sexual  255.000000  255.000000  Default  28.0000  28.0000
383      violencia  415.000000  415.000000  Default  27.0000  27.0000
520       imputado  490.000000  490.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
68       comunidad   20.056923  128.806115   Topic5  -5.6842   0.5968
366           tema   17.889321   63.920931   Topic5  -5.7986   1.1831
758       lesiones   21.947753  291.172343   Topic5  -5.5941  -0.1287
383      violencia   18.140708  415.758881   Topic5  -5.7846  -0.6754
791      sentencia   16.382788  153.154989   Topic5  -5.8866   0.2213

[366 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1812      3  0.974021      2014
4         1  0.036722      2020
4         2  0.018361      2020
4         3  0.009181      2020
4         4  0.697727      2020
...     ...       ...       ...
223       5  0.011864   víctima
468       1  0.202806  víctimas
468       2  0.185422  víctimas
468       3  0.266545  víctimas
468       4  0.336078  víctimas

[613 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 4, 2, 1])